___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 8: Unsupervised learning

## Lab work 8



#### Мета: 

* навчитися використовувати моделі без учителем

### Завдання 1:

Опрацюйте файл `Groceries.csv`. Використовуючи алгоритм асоціативних зв'язків, знайдіть:
- зробіть описову статистику даного файлу
- ведіть топ-3 продукти, що продаються
- виведіть топ-2 пари продуктів, що продаються
- придумайте, як отриману інформацію можна використатти (опишіть в 3-4 реченнях)

In [37]:
import pandas as pd

df = pd.read_csv('groceries.csv')


### Stats

In [91]:
print('###Columns:')
print(df.columns)
print('###Shape:')
print(df.shape)
print('###Describe:')
print(df.describe())

###Columns:
Index(['Item(s)', 'Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6',
       'Item 7', 'Item 8', 'Item 9', 'Item 10', 'Item 11', 'Item 12',
       'Item 13', 'Item 14', 'Item 15', 'Item 16', 'Item 17', 'Item 18',
       'Item 19', 'Item 20', 'Item 21', 'Item 22', 'Item 23', 'Item 24',
       'Item 25', 'Item 26', 'Item 27', 'Item 28', 'Item 29', 'Item 30',
       'Item 31', 'Item 32'],
      dtype='object')
###Shape:
(9835, 33)
###Describe:
           Item(s)
count  9835.000000
mean      4.409456
std       3.589385
min       1.000000
25%       2.000000
50%       3.000000
75%       6.000000
max      32.000000


In [39]:
df['Items'] = df.iloc[:, 1:].apply(lambda x: ' , '.join(x.dropna()), axis=1)

clean_items = df['Items'].str.split(',').explode().str.strip()

print('###Top 3 products:')
print(clean_items.value_counts().head(3))


###Top 3 products:
whole milk          2513
other vegetables    1903
rolls/buns          1809
Name: Items, dtype: int64


In [94]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

data = df.values.tolist()
data = [[x for x in row if str(x) != 'nan'] for row in data]

# at the first index data has length of all items, we want to remove it
data = [row[1:] for row in data]
# print(data)
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df_formatted = pd.DataFrame(te_ary, columns=te.columns_)

df_formatted.to_csv('groceries_formatted.csv', index=False)

# max_len=2 in order to get pairs, most products we already had without any ML
frequent_itemsets = apriori(df_formatted, min_support=0.05, use_colnames=True, max_len=5)
# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

print("Most Frequent Itemsets:")
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
print(frequent_itemsets[frequent_itemsets['length'] > 1])
frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)

# Print association rules
print("\nAssociation Rules:")
rules.to_csv('association_rules.csv', index=False)
rules.head()

Most Frequent Itemsets:
     support                        itemsets  length
28  0.074835  (other vegetables, whole milk)       2
29  0.056634        (rolls/buns, whole milk)       2
30  0.056024            (yogurt, whole milk)       2

Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(other vegetables),(whole milk),0.193493,0.255516,0.074835,0.386758,1.513634,0.025394,1.214013,0.420750
1,(whole milk),(other vegetables),0.255516,0.193493,0.074835,0.292877,1.513634,0.025394,1.140548,0.455803
2,(rolls/buns),(whole milk),0.183935,0.255516,0.056634,0.307905,1.205032,0.009636,1.075696,0.208496
3,(whole milk),(rolls/buns),0.255516,0.183935,0.056634,0.221647,1.205032,0.009636,1.048452,0.228543
4,(yogurt),(whole milk),0.139502,0.255516,0.056024,0.401603,1.571735,0.020379,1.244132,0.422732


Із результатів моделі ми бачимо що самий популярний продукт це пастеризоване молоко

Воно часто попадається в парі з овочами, булочками та йогуртами.
Можна було би проводити експеримент далі, та перевіряти самі популярніші продукти в парі без молока. Але у такому випадку сильно малі метрики support, lift і іншого.

Мені не хоітлося вибирати інщі продукти, бо по моїм відчуттям у них дуже низькі метрики, і вони не будуть показовими.

Так-як завдання було вивести пари 2-х продуктів то ось висновок:
Самі топові продукти це:
- Овочі + Пастеризоване молоко - тут найбільша підтримка
- Пастеризоване молоко + булочки
- Пастеризоване молоко + йогурти

З цього слідує:
Можна робити акції, наприклад продавати молоко + якись продукт який вже давно лежить на складі, який треба продати. Так-як людям потрібне молоко вони використають знижку на нього + куплять продукт від якого треба здихатися.

Такий трюк також можна спробувати з овочами, булочками та йогуртами.

Також як варіант можна побачити які марки молока купують найчастіше, і замовляти їх більше, або навіть робити акції на них.
Це саме можна зробит из овочами, подивитися які купують найбільше і моєливо відмовитися від тих які найменьше.

Ще 1 висновок, так-як люди полюбляють булочки, а всі люди люблять щоб булочки були свіжими, можна робити рекламу, наприклад що свіжі булочки продаються в стільки то годин, і щоб люди в цей час приходили частіше щоб купити свіже, а у вечері продавати булочки по меньшій ціні, щоб не викидати їх.










### Завдання 2:

Завантажте файл https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python
 
- Спробуйте провести кластеризацію, тобто виділити цільові групи, для яких можна впроваджувати певні маркетингові акції
- Опишіть отримані результати